In [25]:
import numpy as np
from gurobipy import Model, GRB, quicksum
#import matplotlib.pyplot as plt
#from tabulate import tabulate

In [26]:
mdl = Model('BEP')
mdl.setParam("TimeLimit", 15*60)

Set parameter TimeLimit to value 900


In [27]:
fileName = "../SantoDomingo/30-buses.txt"
#fileName = "instancias/InstanceBEP-8-40-20-20_0.txt"
f = open(fileName, 'r')

firstLine = f.readline().strip("\n").split(" ")
b = int(firstLine[0].split(":")[0])
c = int(firstLine[1])

secondLine = f.readline().strip("\n").split(":")
y = int(secondLine[0])
Y = list(map(int, secondLine[1].strip().split(" ")))

thirdLine = f.readline().strip("\n").split(":")
p = int(thirdLine[0])
personasTotal = int(thirdLine[1])
P = list(map(int, thirdLine[2].strip().split(" ")))

fourthLine = f.readline().strip("\n").split(":")
s = int(fourthLine[0])
sumaCapacidadesRefugios = int(fourthLine[1])
S = list(map(int, fourthLine[2].strip().split(" ")))

y_range = range(y)
p_range = range(y, y + p)
s_range = range(y + p, y + p + s)

f.readline() # Empty line
distanciasEstacionesPuntosEncuentros = []

for _ in range(y):
    distanciasEstacionPuntosEncuentros = f.readline().strip("\n").split(":")
    distanciasEstacionesPuntosEncuentros.append(distanciasEstacionPuntosEncuentros[1].strip().split(" "))

distanciasEstacionesPuntosEncuentros = np.array(distanciasEstacionesPuntosEncuentros).astype(int)

f.readline() # Empty line
distanciasPuntosEncuentrosRefugios = []

for _ in range(p):
    distanciasPuntoEncuentroRefugios = f.readline().strip("\n").split(":")
    distanciasPuntosEncuentrosRefugios.append(distanciasPuntoEncuentroRefugios[1].strip().split(" "))

distanciasPuntosEncuentrosRefugios = np.array(distanciasPuntosEncuentrosRefugios).astype(int)

In [28]:
N = [i for i in range(y + p + s)] # Nodos
A = [(i, j) for i in range(y) for j in p_range if i != j] # Arcos partida -> punto de encuentro
A += [(i, j) for i in p_range for j in s_range if i != j] # Arcos punto de encuentro -> refugio
A += [(j, i) for i in p_range for j in s_range if i != j] # Arcos refugio -> punto de encuentro

In [29]:
T = mdl.addVar(vtype=GRB.INTEGER, name="T")
mdl.setObjective(T, GRB.MINIMIZE)

t_evac = 10

In [30]:
tau = {}
for i, j in A:
    if i in y_range:
        tau[(i, j)] = distanciasEstacionesPuntosEncuentros[i][j - y]
    else:
        if (j, i) in tau:
            tau[(i, j)] = tau[(j, i)]
        else:
            tau[(i, j)] = tau[(j, i)] = distanciasPuntosEncuentrosRefugios[i - y][j - p - y] # Tiempo de viaje

In [31]:
# x_{ij}^{vt}: x es 1 si en el viaje t el bus m va de i a j, 0 en caso contrario
x = mdl.addVars(A, b, t_evac, vtype=GRB.BINARY)

In [32]:
# b_{j}^{vt}: numero de evacuados desde el nodo j que serán transportados por el bus m (o si j es un refugio, liberados) después del viaje t
b_ = mdl.addVars(N, b, t_evac, vtype=GRB.INTEGER)

In [33]:
for t1 in range(1, t_evac, 2): # No pueden haber viajes desde un refugio a un punto de encuentro en un viaje impar.
    for m in range(b):
        for i, j in A:
            if i in s_range:
                mdl.addConstr(x[i, j, m, t1] == 0)

for t1 in range(2, t_evac, 2): # No pueden haber viajes desde un punto de encuentro a un refugio en un viaje par.
    for m in range(b):
        for i, j in A:
            if i in p_range:
                mdl.addConstr(x[i, j, m, t1] == 0)

for m in range(b): # 2) El tiempo de evacuación resultante debe ser mayor o igual al costo máximo de cualquier viaje.
    sumaTotal = 0
    for t1 in range(t_evac):
        for i, j in A:
            sumaTotal += tau[i, j] * x[i, j, m, t1]
    mdl.addConstr(sumaTotal <= T)

for m in range(b): # 3) se asegura que un bus que viaja al nodo j en el viaje t deja el nodo j en el viaje t + 1. Desde un punto de partida a un punto de encuentro a un refugio
    for t1 in range(1):
        for j in p_range:
            otra = 0
            suma = 0
            for i in y_range:
                otra += x[i, j, m, t1]
            for k in s_range:
                suma += x[j, k, m, t1 + 1]
            mdl.addConstr(otra == suma)

for m in range(b): # 3) se asegura que un bus que viaja al nodo j en el viaje t deja el nodo j en el viaje t + 1. Desde un refugio a un punto de encuentro a un refugio
    for t1 in range(2, t_evac - 1, 2):
        for j in p_range:
            otra = 0
            suma = 0
            for i in s_range:
                otra += x[i, j, m, t1]
            for k in s_range:
                suma += x[j, k, m, t1 + 1]
            mdl.addConstr(otra == suma)

for m in range(b): # 4) no se requiere que el bus deje el refugio s
    for t1 in range(1, t_evac - 1, 2):
        for j in s_range:
            otra = 0
            suma = 0
            for i in p_range:
                otra += x[i, j, m, t1]
            for k in p_range:
                suma += x[j, k, m, t1 + 1]
            mdl.addConstr(otra >= suma)

for m in range(b): # 5) Un bus solamente puede hacer un viaje a la vez.
    for t1 in range(t_evac - 1):
        suma = 0
        for i, j in A:
            suma += x[i, j, m, t1]
        mdl.addConstr(suma <= 1)
        
for i in y_range: # 6) Cada bus debe partir desde la estacion en su primer viaje.
    offset = sum(Y[:i])
    for m in range(Y[i]):
        suma = 0
        for j in p_range:
            suma += x[i, j, m + offset, 0]
        mdl.addConstr(suma == 1)

for m in range(b): # 7) Si un bus no sale de su estacion al comienzo de la evacuacion, no sale tampoco en viajes posteriores.
    for t1 in range(1, t_evac):
        for y1, j in A:
            if y1 in y_range:
                mdl.addConstr(x[y1, j, m, t1] == 0)

for m in range(b): # 8) No se permite que el ultimo viaje de un bus termine en un punto de encuentro.
    for i, p1 in A:
        if p1 in p_range:
            mdl.addConstr(x[i, p1, m, t_evac - 1] == 0)

for t1 in range(t_evac): # 9) Un bus solo puede recoger a los evacuados desde el nodo j si ha viajado al nodo j.
    for m in range(b):
        for j in N:
            suma = 0
            for i in N: 
                if (i, j) in A:
                    suma += x[i, j, m, t1]
            if (suma):
                mdl.addConstr(b_[j, m, t1] == c * suma)

for t2 in range(t_evac): # 10) Capacidad del bus
    for m in range(b):
        suma1 = 0
        for j in p_range:
            for l in range(t2 + 1):
                suma1 += b_[j, m, l]
        suma2 = 0
        for k in s_range:
            for l in range(t2 + 1):
                suma2 += b_[k, m, l]
        mdl.addConstr(0 <= suma1 - suma2)
        mdl.addConstr(suma1 - suma2 <= c)

for j in range(s): # 11) Capacidad del refugio.
    suma = 0
    for m in range(b):
        for t1 in range(t_evac):
            suma += b_[j + y + p, m, t1]
    mdl.addConstr(suma <= S[j])

for j in range(p): # 12) Todos los evacuados deben ser recogidos de los puntos de encuentro.
    suma = 0
    for m in range(b):
        for t1 in range(t_evac):
            suma += b_[j + y, m, t1]
    mdl.addConstr(suma == P[j])

for m in range(b): # 13) Todos los evacuados deben ser llevados a los refugios.
    suma1 = 0
    for p1 in p_range:
        for t1 in range(t_evac):
            suma1 += b_[p1, m, t1]
    suma2 = 0
    for s1 in s_range:
        for t2 in range(t_evac):
            suma2 += b_[s1, m, t2]
    mdl.addConstr(suma1 == suma2)

In [34]:
mdl.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license 2543337 - for non-commercial use only - registered to re___@usm.cl
Optimize a model with 20922 rows, 29401 columns and 156690 nonzeros
Model fingerprint: 0xdca24021
Variable types: 0 continuous, 29401 integer (25200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]


  RHS range        [1e+00, 2e+02]
Presolve removed 17100 rows and 18120 columns
Presolve time: 0.21s
Presolved: 3822 rows, 11281 columns, 53050 nonzeros
Variable types: 0 continuous, 11281 integer (11250 binary)

Root relaxation: infeasible, 432 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 1 nodes (432 simplex iterations) in 0.29 seconds (0.23 work units)
Thread count was 12 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [35]:
if mdl.status == GRB.OPTIMAL or mdl.status == GRB.TIME_LIMIT:
    for m in range(b):
        print(f"Bus {m + 1}:", end=" ")
        tiempo = 0
        for t1 in range(t_evac):
            for i, j in A:
                if x[i, j, m, t1].x > 0:
                    if t1 == 0:
                        print(f"{i + 1} -> {j + 1 - y}", end=" | ")
                    else:
                        if t1 % 2 == 1:
                            print(f"{i + 1 - y} -> {j + 1 - y - p}", end=" | ")
                        #else:
                        #    print(f"{i + 1} -> {j + 1 - y}", end=" | ")
                    #print(f"El bus {m} viaja del nodo {i} al nodo {j} en el instante {t1}.")
                    tiempo += tau[i, j]
        print(f"Time: {tiempo}")
        #print("\nEl bus", m, "se demora", tiempo, "minutos en evacuar a todas las personas.\n\n")
    print()
    for m in range(b):
        for t1 in range(t_evac):
            for j in N:
                if b_[j, m, t1].x > 0:
                    if t1 == 0:
                        print(f"El bus {m + 1} recoge {b_[j, m, t1].x} personas en el punto de encuentro {j - y + 1} en el instante {t1 + 1}.")
                    else:
                        if t1 % 2 == 1:
                            print(f"El bus {m + 1} deja {b_[j, m, t1].x} personas en el refugio {j - y - p + 1} en el instante {t1 + 1}.")
                        else:
                            print(f"El bus {m + 1} recoge {b_[j, m, t1].x} personas en el punto de encuentro {j - y + 1} en el instante {t1 + 1}.")
        print()
else:
    print("No se encontró una solución óptima ni factible.")


No se encontró una solución óptima ni factible.


In [36]:
if mdl.status == GRB.OPTIMAL or mdl.status == GRB.TIME_LIMIT:
    print("Solución óptima encontrada:", T.X)
    print("Execution time:", mdl.Runtime, "seconds")